# Source location privacy in Cyber-physical systems

- 论文算法设计
    - 骨干网络构建
    - 虚假消息广播

In [1]:
from BaseSLPinCPS import *

# 虚假源调度算法

In [67]:
class cpstopoFakeScheduling(commonPM):
    """
    虚假源调度算法，包含两个阶段：
    1）骨干网络构建
    2）虚假消息调度
    --------------
    用例：
    # 生成对象
    fs = cpstopoFakeScheduling(G=cpsNetwork(nodeNumber=1000, areaLength=100, initEnergy=1e8, radius=10),\
                               Tmax=100, c_capture=1e-3)
    # 虚假源调度算法主函数
    fs.fakeScheduling()
    # 生成骨干网络
    fs.backbonePlot()
    # 结果绘制
    fs.plotDelayandConsumption()
    --------------
    变量成员：
    G          = 网络
    Tmax       = 最大周期数
    
    C_Capture  = 被捕获率阈值，1e-4
    C_alpha    = 超参数 alpha，0.5
    C_Beta     = 超参数 beta，0.5
    
    sink       = 基站编号
    sink_pos   = 基站位置，(0,0)
    source     = 源节点编号
    source_pos = 源节点位置，(0.45*20, 0.45*20)
    attacker   = 攻击者
    
    backbone   = 骨干网络
    safety     = 安全周期数
    listDelay  = 每周期的时延
    listEnergyConsumption = 每周期的传输能耗
    listFakeSource        = 每周期源节点序列，用节点编号表示
    --------------
    方法成员：
    __init__(G, Tmax, c_capture, c_alpha, c_beta, sink_pos, source_pos)
    display()
    generateSINKandSOURCE()
    deployAttacker(node)
    calculate2Distance(node1, node2)
    isConnected()
    searchDeepFirst(u, former, bestBackbone, target, likelihood, maxStep)
    generateBackbone()
    calculateFakeSource(node, Ti)
    delayModel(u, v)
    consumeEnergyModel(node)
    sendSource2Sink(Ti)
    scheduingFakeMessages()
    resultPlot()
    backbonePlot()
    fakeScheduling()     = 虚假源调度主算法入口
    """
    def __init__(self, G = cpsNetwork(nodeNumber=10, areaLength=20, initEnergy=1e6, radius=10), Tmax=1000, \
                 c_capture=1e-4, c_alpha=0.5, c_beta=0.5, sink_pos=(0,0), source_pos=(0.45*20, 0.4*20)):
        self.G = G
        self.Tmax = Tmax
        
        self.C_Capture = c_capture
        self.C_Alpha = c_alpha
        self.C_Beta = c_beta 
        
        self.sink = -1
        self.sink_pos = sink_pos
        self.source = -1
        self.source_pos = source_pos
        self.attacker = cpsAttacker()
        
        self.backbone = []
        self.safety = -1                # 网络安全周期
        self.listDelay = []             # 每个周期打网络时延
        self.listEnergyConsumption = [] # 每个周期的网络能耗
        self.listFakeSource = []        # 每个周期的虚假源部署情况
        
    def initModel(self):
        self.attacker = cpsAttacker()
        
        self.backbone = []
        self.safety = -1                # 网络安全周期
        self.listDelay = []             # 每个周期的网络时延
        self.listEnergyConsumption = [] # 每个周期的网络能耗
        self.listFakeSource = []        # 每个周期的虚假源部署情况
        
    def display(self):
        print "节点总数：", self.G.nodeNumber
        print "正方形区域边长：", self.G.areaLength
        print "节点初始能量：", self.G.initEnergy
        print "最大周期数：", self.Tmax
        print "节点通信半径：", self.G.radius
        print "捕获率阈值：", self.C_Capture
        print "参数 alpha：", self.C_Alpha
        print "参数 beta：", self.C_Beta
        print "sink 编号：", self.sink
        print "source 编号：", self.source
    
    def generateSINKandSOURCE(self):
        """
        增加 sink 和 source，并更新邻接矩阵元素个数(+2)
        其中，
        sink 位于区域中心，(0,0)
        source 位于区域右上角 (0.45*areaLength, 0.45*areaLength)
        """
        self.sink = self.G.nodeNumber
        self.source = self.G.nodeNumber+1
        self.G.nodeList.append(cpsNode(identity=self.G.nodeNumber, \
                                       position=self.sink_pos, \
                                       energy=self.G.initEnergy*100., radius=self.G.radius, \
                                       state="SINK"))
        self.G.nodeList.append(cpsNode(identity=self.G.nodeNumber+1, \
                                       position=self.source_pos, \
                                       energy=self.G.initEnergy, radius=self.G.radius, \
                                       state="SOURCE"))
        self.G.adjacentMatrix = np.zeros((self.G.nodeNumber+2, self.G.nodeNumber+2), dtype=np.int8)
        self.G.nodeNumber += 2     # 网络节点数量 +2
        
    def deployAttacker(self, node):
        """
        部署攻击者初始位置，位于 sink
        """
        while self.attacker.trace:
            self.attacker.trace.pop()
        self.attacker.position = node
        self.attacker.trace.append(node.identity)
    
    def calculate2Distance(self, node1, node2):
        "计算两个节点之间的欧式距离"
        x1 = node1.position[0]
        y1 = node1.position[1]
        x2 = node2.position[0]
        y2 = node2.position[1]
        return np.sqrt((x1-x2)**2+(y1-y2)**2)
    
    def isConnected(self):
        "判定一个网络是否连通"
        print "网络连通性判定..."
        numConnected = 0
        
        queue = Queue.Queue()
        visited = np.zeros(self.G.nodeNumber, dtype=np.int8)
        queue.put(self.sink)  # 从 sink 开始广搜整个网络
        visited[self.sink] = 1
        numConnected += 1
        while not queue.empty():
            u = queue.get()
            U = self.G.nodeList[u]
            for V in self.G.nodeList:
                v = V.identity
                if u != v and self.calculate2Distance(U, V) < U.radius:
                    U.adj.add(v)
                    if visited[v] == 0:
                        queue.put(v)
                        visited[v] = 1
                        numConnected += 1
        if numConnected == self.G.nodeNumber:
            return True
        return False
    def searchDeepFirst(self, u, former, bestBackbone, target, likelihood, maxStep):
        """
        深度优先搜索：(前驱节点，已走过的路径，最佳路径，目标值，当前似然，最大步数)
        寻找所有长度为 maxStep 的路径，满足 self.C_Capture
        """
        U = self.G.nodeList[u]
        for v in U.adj:
            V = self.G.nodeList[v]
            if v not in former and len(former)+V.level <= maxStep:#len(former)+1 <= maxStep: # 长度限制
                diff = len(V.adj - U.adj) if u != self.source else len(V.adj)
                former.append(v)
                # 每到一次 sink，更新一次 target 值
                if v == self.sink:
#                     print former, "\nThe capture rate is", V.likelihood
                    if likelihood/(diff+1.0) < target:
                        target = likelihood/(diff+1.0)
                        bestBackbone = former[:]
                    else:
                        pass
                    # 回溯
                    former.pop()
                    return target, bestBackbone
                else:
                    target, bestBackbone = self.searchDeepFirst(v, former, bestBackbone, \
                                                                   target, likelihood/(diff+1.0), maxStep)
                    # 回溯
                    former.pop()
                    # 当 target 被更新以后（即已经到达过 sink），且满足要求，搜索结束
                    if target <= self.C_Capture:
                        return target, bestBackbone
        return target, bestBackbone
                
    
    def generateBackbone(self):
        """
        骨干网络构建算法
        """
        print "构建骨干网络..."       
        queue = Queue.Queue()
        visited = np.zeros(self.G.nodeNumber, dtype=np.int8)
        
        self.G.nodeList[self.sink].level = 0
        queue.put(self.sink)
        visited[self.sink] = 1
        # 计算节点与基站之间最短打跳数
        while not queue.empty():
            u = queue.get()
            visited[u] = 0
            U = self.G.nodeList[u]
            for v in U.adj:
                V = self.G.nodeList[v]
                if V.level == -1 or U.level + 1 < V.level:
                    V.level = U.level + 1
                    if visited[v] == 0:
                        queue.put(v)
                        visited[v] = 1
                        
        sourceLikelihood = 1.  # 被捕获似然的目标值
        sourceLevel = self.G.nodeList[self.source].level+1  # 记录源节点与基站的跳数
        former = [] # 传输路径
        
        
        # 迭代搜索满足被捕获似然打的骨干网络
        while sourceLikelihood >= self.C_Capture:
            # 评估源节点被捕获似然（深搜直至发现满意骨干网络）
            start_clock = time.clock()
            target, bestBackbone = self.searchDeepFirst(self.source, [self.source], [], 1.0, 1.0, sourceLevel)
            finish_clock = time.clock()
            print "It takes", (finish_clock - start_clock), 's on the construction of backbone '
            # 记录似然值
            sourceLikelihood = target
            if sourceLikelihood == -1:   # 未找到 sink 节点
                sourceLikelihood = 1.
            else:
                pass
            # 迭代更新跳数长度
            sourceLevel += 1
            if sourceLevel >= self.G.nodeNumber:
                break 
        # 记录骨干网络
        print "The sourceLikelihood is", sourceLikelihood
        if sourceLikelihood < self.C_Capture:
            self.backbone = bestBackbone
            print "骨干网络：", bestBackbone

    def calculateFakeSource(self, node, Ti):
        """
        计算节点间通信的概率，并返回 True/False 值
        PS: sink, source, backbone 不会成为 fake source
        """
        if node.state == "SINK" or node.state == "SOURCE":
            return False
        B = set(self.backbone)
        if node.identity in B:
            return False
        
        # I_v_i^B
        numB = 0
        for v in node.adj:
            if v in B:
                numB += 1
        # rank_v_i^E
        rankEV = -1
        if Ti == 1:
            rankEV = 1./len(node.adj)
        else:
            rank = 1
            for v in node.adj:
                V = self.G.nodeList[v]
                if V.energy > node.energy:  # 计算节点能耗排名，能耗多的排名靠前
                    rank += 1
            rankEV = rank*1./len(node.adj)
        # I_v_i^C
        numC = 0
        if Ti == 1:
            numC = 0
        else:
            for v in node.adj:
                V = self.G.nodeList[v]
                if V.weight == 1:
                    numC += 1
        # C(p_i^T(l-1))
        CP = 0
        if Ti == 1:
            CP = 0
        else:
            CP = node.weight
        # p_i
        numI = len(node.adj)
        p_i_z = self.C_Alpha*np.exp(numB*1./numI) # 分子
        p_i_m = self.C_Beta*np.exp(1.-rankEV*1./numI) + (1-self.C_Beta)*np.exp(CP-numC*1./numI) # 分母
        p_i = p_i_z/p_i_m # 概率阈值
        # 是否广播
        RAND = np.random.rand()
        if RAND < p_i:
            return True
        else:
            return False
    
    def delayModel(self, u, v):
        """
        估计单个节点，单跳，单位 bit 数据的时延
        """
        tuv = 100.       # 无干扰,bit,接收时间 ns
        pe = 0           # 环境的噪声功率
        guvl = 9.488e-5  # 无线设备参数，d < d0
        guvh = 5.0625    # 无线设备参数，d >= d0
        d0 = 231         # 跨空间距离阈值
        pt = 10e0        # 节点发送功率
        U = self.G.nodeList[u]
        V = self.G.nodeList[v]
        pv = 0           # 接收节点干扰功率
        for neighbor in V.adj:
            Neighbor = self.G.nodeList[neighbor]
            if neighbor == u or (Neighbor.state == "NORMAL" and Neighbor.weight == 0):
                continue
            else:
                d = self.calculate2Distance(Neighbor, V)
                if d < d0:
                    puv = (guvl*pt)/(d**2)
                else:
                    puv = (guvh*pt)/(d**4)
                pv += puv
        puv = 0          # u->v 接收功率
        d = self.calculate2Distance(U, V)
        if d < d0:
            puv = (guvl*pt)/(d**2)
        else:
            puv = (guvh*pt)/(d**4)
        ruv = 0          # 信干噪比
        if pv > 1e-32:
            ruv = puv/(pe+pv)
        else:
            ruv = 20.
        # 范围单跳，单位 bit 时延
        return tuv/(1.-np.exp(ruv*(-0.5)))
         
        
    def consumeEnergyModel(self, node):
        """
        估计单个节点，单位 bit 数据，的能耗
        """
        d0 = 231         # 跨空间距离阈值
        d = node.radius  # 节点广播半径
        Eelec = 50.      # 单位 bit 数据能耗
        c_fs = 10e-3     # 自由空间下单位放大器能耗
        C_mp = 0.0013e-3 # 存在多径损耗打单位放大器能耗
        # 接收能耗
        rE = 0           # 单位数据接收能耗
        for neighbor in node.adj:
            Neighbor = self.G.nodeList[neighbor]
            if (Neighbor.weight == 1) or (neighbor in set(self.backbone)):
                rE += Eelec
        # 发送能耗
        tE = 0           # 单位数据发送能耗
        if (node.weight == 1) or (node in set(self.backbone)):
            tE += Eelec
            if d < d0:
                tE += c_fs*(d**2)
            else:
                tE += c_mp*(d**4)
        return rE + tE
        
    
    def sendSource2Sink(self, Ti):
        """
        一个周期内容事件：
        1）主要事件：源节点发送消息，并通过骨干网络传输至基站
        2）攻击者移动模型
        3）虚假源广播
        更新的参数：
        1）节点剩余能量
        2）攻击者位置
        返回值：
        1）源节点是否被捕获
        2）源节点消息上传到基站的时间花费
        3）网络能耗
        """
        packetSize = 500*8  # 单次数据包大小 bit
        # 路由时延
        delayTi = 0
        u = -1
        for i,v in enumerate(self.backbone):
            if i == 0:
                u = v
            else:
                delayTi += self.delayModel(u, v)*packetSize
                u = v
        # 网络能耗
        energyTi = 0
        maxEC = 0
        for node in self.G.nodeList:
            ec = self.consumeEnergyModel(node)
            ec *= packetSize
            energyTi += ec
            maxEC = max(maxEC, ec)
            node.energy -= ec
        # 攻击者移动
        self.attacker.traceBack(self.backbone, self.G)
        # 源节点是否被捕获
        flag = False
        if self.attacker.position.identity == self.source:
            flag = True
        else:
            flag = False
        return flag, delayTi, maxEC
    
    def scheduingFakeMessages(self):
        """
        虚假消息调度
        """
        listDelay = []
        listEnergyConsumption = []
        safety = -1
        for Ti in range(1, self.Tmax+1):
            if Ti % 500 == 0:
                print Ti
            elif Ti % 50 == 0:
                print Ti,
            state = {}
            for node in self.G.nodeList:
                state[node.identity] = self.calculateFakeSource(node, Ti)
            # update 节点权重，1：fake，0：not fake
            for node in self.G.nodeList:
                node.weight = 1 if state[node.identity] else 0
            self.listFakeSource.append([node.identity for node in self.G.nodeList if node.weight == 1])
            # 源节点发送消息给基站的事件
            flag, delayTi, energyTi = self.sendSource2Sink(Ti)
            # 保存每轮的记录
            listDelay.append(delayTi)
            listEnergyConsumption.append(energyTi)
            if flag:
                safety = Ti
                break
            elif Ti == self.Tmax:
                safety = self.Tmax
            else:
                pass
        return safety, listDelay, listEnergyConsumption
    
    def plotDelayandConsumption(self):
        """
        曲线：
        1）每轮的网络时延
        2）每轮的能耗
        """
        plt.figure(figsize=(15,6))
        plt.subplot(2,1,1)
        plt.semilogy(np.array(self.listDelay)/1e9)
        plt.title("Delay")
        plt.subplot(2,1,2)
#         plt.semilogy(np.array(self.listEnergyConsumption)/self.G.nodeNumber)
        plt.semilogy(np.array(self.listEnergyConsumption))
        plt.title("Consumption")
        plt.show()
    
    def backbonePlot(self):
        """
        骨干网络绘制
        """
        ax = plt.figure(figsize=(5,5))
        temp_x = []
        temp_y = []
        for i in range(self.G.nodeNumber):
            if i in self.backbone:
                continue
            temp_x.append(self.G.nodeList[i].position[0])
            temp_y.append(self.G.nodeList[i].position[1])
        plt.plot(temp_x, temp_y, 'yp')
        # 骨干网络
        u = -1
        for i,v in enumerate(self.backbone):
            if i == 0:
                u = v
                continue
            else:
                U = self.G.nodeList[u]
                V = self.G.nodeList[v]
                x = [U.position[0], V.position[0]]
                y = [U.position[1], V.position[1]]
                plt.plot(x, y, 'k')
                u = v
        # 骨干节点
        temp_x = []
        temp_y = []
        for i in self.backbone:
            temp_x.append(self.G.nodeList[i].position[0])
            temp_y.append(self.G.nodeList[i].position[1])
        plt.plot(temp_x, temp_y, 'rs')
        plt.axis("equal")
        plt.show()
    
    def fakeScheduling(self):
        """
        虚假源调度算法的主函数
        1）生成 sink 和 source
        2）网络连通性判定，直至网络连通
        3）骨干网络构建
        4）虚假源调度及事件模拟
        其中，
        1）时延，单位 ns,纳秒
        2）能耗，单位 nj,纳焦
        """
        self.generateSINKandSOURCE()
        flag = self.isConnected() # 更新各节点邻居
        if not flag:
            print "网络不连通"
            return
        else:
            print "网络连通"
        self.deployAttacker(self.G.nodeList[self.sink]) # 部署攻击者位置
        self.generateBackbone() # 生成骨干网络
        safety, listDelay, listEnergyConsumption = self.scheduingFakeMessages() # 虚假源调度与网络路由事件
        # 输出的指标
        self.safety = safety
        self.listDelay = listDelay
        self.listEnergyConsumption = listEnergyConsumption
        print "\nThe safety is", self.safety

In [74]:
if __name__ == '__main__':
    fs = cpstopoFakeScheduling(G=cpsNetwork(nodeNumber=800, areaLength=150, initEnergy=1e8, radius=10), \
                                Tmax=1000, c_capture=1e-8, c_alpha=0.2, c_beta=0.5, \
                                sink_pos=(-0.4*200, -0.4*200), source_pos=(0.4*200, 0.4*200))


    fs.generateSINKandSOURCE()
    flag = fs.isConnected() # 更新各节点邻居
    if not flag:
        print "网络不连通"
    else:
        print "网络连通"
    while not flag:
        fs = cpstopoFakeScheduling(G=cpsNetwork(nodeNumber=800, areaLength=150, initEnergy=1e8, radius=10),\
                                Tmax=1000, c_capture=1e-8, c_alpha=0.2, c_beta=0.5, \
                                sink_pos=(-0.4*200, -0.4*200), source_pos=(0.4*200, 0.4*200))
        fs.generateSINKandSOURCE()
        flag = fs.isConnected() # 更新各节点邻居

#     fs.fakeScheduling()
    result_dict = dict()
    for ca in np.linspace(0.2, 1, 5):
        for cb in np.linspace(0, 1, 6):
            fs.initModel()
            fs.c_alpha = ca
            fs.c_beta = cb
            print 'Alpha = %.2f, Beta = %.2f' % (ca, cb)
            experiment_energy = []
            experiment_delay = []
            experiment_safety = []
            if flag:
                for i in range(10):
                    print '\nExperiment %d' % (i+1)
                    fs.deployAttacker(fs.G.nodeList[fs.sink]) # 部署攻击者位置
                    fs.generateBackbone() # 生成骨干网络
                    safety, listDelay, listEnergyConsumption = fs.scheduingFakeMessages() # 虚假源调度与网络路由事件
                    # 输出的指标
                    fs.safety = safety
                    fs.listDelay = listDelay
                    fs.listEnergyConsumption = listEnergyConsumption
                    print "\nThe safety is", fs.safety
                    experiment_energy.append(max([1e8-node.energy for node in fs.G.nodeList \
                                                                  if node.identity != fs.sink]))
                    experiment_delay.append(max(fs.listDelay))
                    experiment_safety.append(fs.safety)
                result_dict[(ca, cb)] = (np.mean(experiment_energy),\
                                         np.mean(experiment_delay),\
                                         np.mean(experiment_safety))

        #         fs.backbonePlot()
        #         fs.plotDelayandConsumption()
        #         # 每轮的虚假源节点数量
        #         a = [len(x) for x in fs.listFakeSource]
        #         plt.figure(figsize=(15,3))
        #         plt.plot(a)
        #         plt.ylabel('The number of fake source')
        #         plt.show()

 网络连通性判定...
网络不连通
网络连通性判定...
网络连通性判定...
Alpha = 0.20, Beta = 0.00

Experiment 1
构建骨干网络...
It takes 0.000125999999909 s on the construction of backbone 
The sourceLikelihood is 3.34598517561e-27
骨干网络： [801, 60, 577, 226, 0, 513, 669, 592, 232, 684, 790, 117, 124, 166, 784, 264, 230, 227, 621, 637, 734, 1, 466, 561, 346, 121, 705, 228, 217, 349, 654, 800]
50 100 150 200 250 300 350 400 450 500
550 600 650 700 750 
The safety is 780

Experiment 2
构建骨干网络...
It takes 0.000135999999998 s on the construction of backbone 
The sourceLikelihood is 3.34598517561e-27
骨干网络： [801, 60, 577, 226, 0, 513, 669, 592, 232, 684, 790, 117, 124, 166, 784, 264, 230, 227, 621, 637, 734, 1, 466, 561, 346, 121, 705, 228, 217, 349, 654, 800]
50 100 150 200 250 300 350 400 450 500
550 600 650 700 750 800 850 900 950 1000

The safety is 1000

Experiment 3
构建骨干网络...
It takes 0.000121000000036 s on the construction of backbone 
The sourceLikelihood is 3.34598517561e-27
骨干网络： [801, 60, 577, 226, 0, 513, 669, 592, 232,

In [77]:
# import pandas as pd

# df = pd.DataFrame(result_dict, index=['EnergyConsumption', 'Delay', 'Safety'])

# df.to_csv('alpha-beta-1-10-800-150-1e8-10-1000.csv')

# from mpl_toolkits.mplot3d import Axes3D
# from matplotlib import cm

# X = np.linspace(0.2, 1, 5)
# Y = np.linspace(0, 1, 6)
# X, Y = np.meshgrid(X, Y)
# Z1, Z2, Z3 = np.zeros(X.shape), np.zeros(X.shape), np.zeros(X.shape)
# for key, value in result_dict.items():
#     rx, ry = key[0], key[1]
#     flag = False
#     for i in range(X.shape[0]):
#         if flag:
#             break
#         for j in range(X.shape[1]):
#             if flag:
#                 break
#             x = X[i,j]
#             y = Y[i,j]
#             if abs(rx-x)<0.1 and abs(ry-y)<0.1:
#                 Z1[i,j], Z2[i,j], Z3[i,j] = value[0], value[1], value[2]
#                 flag = True

# def plot3dZ(X, Y, Z, flag, zlabel):
#     fig = plt.figure()
#     ax = fig.gca(projection='3d')
#     surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.coolwarm, lw=0.5)
#     ax.set_xticks([0.2, 0.6, 1.])
#     ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8])
#     ax.set_xlabel(r'$\alpha$')
#     ax.set_ylabel(r'$\beta$')
#     ax.set_zlabel(zlabel)
#     if flag:
#         ax.set_zscale('log')
#     ax.view_init(30, 40)
#     fig.colorbar(surf, shrink=0.5, aspect=5)
    
# plot3dZ(X, Y, Z1, True, 'Energy Consumption (nJ)')
# plt.savefig('ec.png', dpi=300)
# plt.show()

# plot3dZ(X, Y, Z2, True, 'Delay (ns)')
# plt.savefig('delay.png', dpi=300)
# plt.show()

# plot3dZ(X, Y, Z3, True, 'Safety (cycle)')
# plt.savefig('safety.png', dpi=300)
# plt.show()